In [3]:
%pylab inline
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
import tensorflow as tf

Populating the interactive namespace from numpy and matplotlib


In [4]:
DATA_DIR = os.path.dirname(os.getcwd())
data = np.loadtxt(os.path.join(DATA_DIR, "test1.csv"), dtype=np.int, delimiter=",")
datat = np.loadtxt(os.path.join(DATA_DIR, "test2.csv"), dtype=np.int, delimiter=",")

In [5]:
def reformat(data):
    x = data[:,1:]
    classes = data[:,0].astype(np.int)
    K = max(classes)+1
    N = x.shape[0]
    y = np.zeros((N,K))
    y[np.arange(N),classes] = 1
    return x, y, classes

In [6]:
X, Y, C = reformat(data)

In [7]:
Xt, Yt, Ct = reformat(datat)

In [8]:
pca = PCA(n_components=200)
pca.fit(X)
Xnew = pca.transform(X)
Xtnew = pca.transform(Xt)

In [12]:
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(SVC(kernel='rbf'), param_grid)
clf = clf.fit(Xnew, C)

In [13]:
best = clf.best_estimator_
preds = best.predict(Xtnew)
error = 1 - (preds == Ct).sum()/ float(len(preds))
print error

0.749791028142


In [15]:
M = 20
N = Xnew.shape[1]
x = tf.placeholder("float", [None, N])
W1 = tf.Variable(tf.random_uniform([N, M], -0.5, 0.5))
l1 = tf.nn.sigmoid(tf.matmul(x, W1))
W2 = tf.Variable(tf.random_uniform([M, 7], -0.5, 0.5))
y = tf.nn.softmax(tf.matmul(l1, W2))

In [16]:
Y.shape

(3589, 7)

In [17]:
y_ = tf.placeholder("float", [None, 7])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.0001).minimize(cross_entropy)
init = tf.initialize_all_variables()

In [18]:
sess = tf.Session()
sess.run(init)
for i in range(5000):
#     idx = i % N
#     sess.run(train_step, feed_dict={x: Xnew[idx:idx+1], y_: Y[idx:idx+1]})
    sess.run(train_step, feed_dict={x: Xnew, y_: Y})

In [19]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [20]:
sess.run(accuracy, feed_dict={x: Xtnew, y_: Yt})

0.29674003